In [1]:
!pip install scikit-fuzzy

   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/920.8 kB ? eta -:--:--
   ---------------------- ----------------- 524.3/920.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------- ----- 786.4/920.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 920.8/920.8 kB 870.1 kB/s  0:00:00


In [ ]:
# ===========================================
# UNIFIED FUZZY EXPERT SYSTEM – EARTHQUAKE DETECTION
# ===========================================

# If running in a notebook for the first time:
# !pip install -q scikit-fuzzy ipywidgets

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# -------------------------------
# Utilities
# -------------------------------
def leer_float(prompt, minimo, maximo, default=None, allow_blank=False):
    """Read a float in [minimo, maximo]. If allow_blank and user sends empty, returns default."""
    while True:
        try:
            raw = input(prompt)
            if allow_blank and raw.strip() == "":
                if default is None:
                    print("Blank not allowed here. Please enter a number.")
                    continue
                return float(default)
            v = float(raw)
            if not (minimo <= v <= maximo):
                print(f"Value out of range [{minimo}–{maximo}]. Try again.")
                continue
            return v
        except Exception:
            print("Invalid input. Enter a valid number.")

def in_notebook():
    try:
        from IPython import get_ipython
        return get_ipython() is not None
    except Exception:
        return False

# -------------------------------
# Variables (Antecedents & Consequent)
# -------------------------------
aceleracion = ctrl.Antecedent(np.arange(0, 1.51, 0.01), 'aceleracion')   # g (0–1.5)
frecuencia  = ctrl.Antecedent(np.arange(0, 20.1, 0.1),  'frecuencia')    # Hz
duracion    = ctrl.Antecedent(np.arange(0, 60.1, 1),    'duracion')      # s
snr         = ctrl.Antecedent(np.arange(0, 30.1, 0.1),  'snr')           # dB
distancia   = ctrl.Antecedent(np.arange(0, 601, 1),     'distancia')     # km
amenaza     = ctrl.Consequent(np.arange(0, 11, 1),      'amenaza')       # 0–10

# Memberships
aceleracion['baja']      = fuzz.trimf(aceleracion.universe, [0.0, 0.0, 0.3])
aceleracion['media']     = fuzz.trimf(aceleracion.universe, [0.2, 0.5, 0.8])
aceleracion['alta']      = fuzz.trimf(aceleracion.universe, [0.6, 1.0, 1.5])
aceleracion['muy_baja']  = fuzz.trapmf(aceleracion.universe, [0.0, 0.0, 0.02, 0.05])
aceleracion['muy_alta']  = fuzz.trapmf(aceleracion.universe, [1.0, 1.2, 1.5, 1.5])
aceleracion['muy_alto']  = fuzz.trapmf(aceleracion.universe, [1.0, 1.2, 1.5, 1.5])  # same shape, new label

frecuencia['baja']       = fuzz.trapmf(frecuencia.universe, [0, 0, 2, 5])
frecuencia['media']      = fuzz.trapmf(frecuencia.universe, [3, 6, 10, 13])
frecuencia['alta']       = fuzz.trapmf(frecuencia.universe, [11, 15, 20, 20])
frecuencia['muy_baja']   = fuzz.gaussmf(frecuencia.universe, 1.0, 0.5)
frecuencia['muy_alta']   = fuzz.smf(frecuencia.universe, 15, 18)

duracion['corta']        = fuzz.trimf(duracion.universe, [0, 0, 10])
duracion['media']        = fuzz.trimf(duracion.universe, [5, 20, 35])
duracion['larga']        = fuzz.trimf(duracion.universe, [30, 60, 60])

snr['pobre']             = fuzz.trapmf(snr.universe, [0.0, 0.0, 4.0, 6.0])
snr['marginal']          = fuzz.trimf( snr.universe, [6.0, 7.75, 9.5])
snr['operacional']       = fuzz.trimf( snr.universe, [9.5, 12.0, 14.0])
snr['aceptable']         = fuzz.trapmf(snr.universe, [8.0, 12.0, 18.0, 22.0])  # neutral shift
snr['buena']             = fuzz.trimf( snr.universe, [14.0, 17.0, 20.0])
snr['excelente']         = fuzz.trapmf(snr.universe, [20.0, 23.0, 30.0, 30.0])

distancia['muy_cercana'] = fuzz.trapmf(distancia.universe, [0, 0, 50, 150])
distancia['cercana']     = fuzz.trimf(distancia.universe, [100, 200, 300])
distancia['moderada']    = fuzz.trimf(distancia.universe, [250, 375, 500])
distancia['lejana']      = fuzz.trapmf(distancia.universe, [450, 550, 600, 600])

amenaza['sin_actividad'] = fuzz.trimf(amenaza.universe, [0, 0, 2])
amenaza['microtemblor']  = fuzz.trimf(amenaza.universe, [1, 3, 5])
amenaza['leve']          = fuzz.trimf(amenaza.universe, [4, 5, 6])
amenaza['moderado']      = fuzz.trimf(amenaza.universe, [5, 7, 8])
amenaza['fuerte']        = fuzz.trimf(amenaza.universe, [7, 9, 10])
amenaza['destructivo']   = fuzz.trimf(amenaza.universe, [9, 10, 10])

# -------------------------------
# Rules
# -------------------------------
rules = []

niveles_acel_core = ['baja', 'media', 'alta']
niveles_freq_core = ['baja', 'media', 'alta']
niveles_dur       = ['corta', 'media', 'larga']
niveles_snr       = ['pobre', 'marginal', 'operacional', 'aceptable', 'buena', 'excelente']

orden_salidas = ['sin_actividad','microtemblor','leve','moderado','fuerte','destructivo']
idx_salida    = {name: i for i, name in enumerate(orden_salidas)}
ajuste_snr    = {'pobre': -1, 'marginal': 0, 'operacional': 0, 'aceptable': 0, 'buena': +1, 'excelente': +1}

def base_output_por(a_base, f_base, d):
    if a_base == 'baja' and f_base == 'alta' and d == 'corta':
        return 'sin_actividad'
    elif a_base == 'baja':
        return 'microtemblor'
    elif a_base == 'media' and f_base == 'alta':
        return 'leve'
    elif a_base == 'media' and f_base == 'media' and d != 'corta':
        return 'moderado'
    elif a_base == 'media':
        return 'leve'
    elif a_base == 'alta' and f_base == 'alta' and d == 'corta':
        return 'microtemblor'
    elif a_base == 'alta' and f_base == 'media' and d == 'corta':
        return 'leve'
    elif a_base == 'alta' and f_base == 'baja' and d == 'larga':
        return 'destructivo'
    elif a_base == 'alta' and f_base == 'baja':
        return 'fuerte'
    else:
        return 'moderado'

def clamp_idx(i):
    return max(0, min(len(orden_salidas) - 1, i))

# Backbone
for a in niveles_acel_core:
    for f in niveles_freq_core:
        for d in niveles_dur:
            base_idx = idx_salida[base_output_por(a, f, d)]
            for s in niveles_snr:
                out = orden_salidas[clamp_idx(base_idx + ajuste_snr[s])]
                rules.append(ctrl.Rule(aceleracion[a] & frecuencia[f] & duracion[d] & snr[s], amenaza[out]))

# Extremes mapped to nearest core
for a_ext, a_core in {'muy_baja': 'baja', 'muy_alta': 'alta', 'muy_alto': 'alta'}.items():
    for f in niveles_freq_core:
        for d in niveles_dur:
            base_idx = idx_salida[base_output_por(a_core, f, d)]
            for s in niveles_snr:
                out = orden_salidas[clamp_idx(base_idx + ajuste_snr[s])]
                rules.append(ctrl.Rule(aceleracion[a_ext] & frecuencia[f] & duracion[d] & snr[s], amenaza[out]))

for f_ext, f_core in {'muy_baja': 'baja', 'muy_alta': 'alta'}.items():
    for a in niveles_acel_core:
        for d in niveles_dur:
            base_idx = idx_salida[base_output_por(a, f_core, d)]
            for s in niveles_snr:
                out = orden_salidas[clamp_idx(base_idx + ajuste_snr[s])]
                rules.append(ctrl.Rule(aceleracion[a] & frecuencia[f_ext] & duracion[d] & snr[s], amenaza[out]))

# Distance-aware rules (Antonio)
rules += [
    ctrl.Rule(aceleracion['baja']  & snr['pobre'], amenaza['sin_actividad']),
    ctrl.Rule(aceleracion['baja']  & distancia['lejana'], amenaza['microtemblor']),
    ctrl.Rule(aceleracion['media'] & frecuencia['alta'] & distancia['lejana'], amenaza['leve']),
    ctrl.Rule(aceleracion['media'] & frecuencia['media'] & distancia['muy_cercana'], amenaza['moderado']),
    ctrl.Rule(aceleracion['media'] & snr['buena'] & distancia['cercana'], amenaza['moderado']),
    ctrl.Rule(aceleracion['alta']  & frecuencia['baja'] & duracion['larga'] &
              distancia['muy_cercana'] & (snr['buena'] | snr['excelente']), amenaza['destructivo']),
    ctrl.Rule(aceleracion['alta']  & frecuencia['baja'] & duracion['media'] &
              (distancia['cercana'] | distancia['moderada']), amenaza['fuerte']),
    ctrl.Rule(aceleracion['alta']  & snr['pobre'], amenaza['microtemblor']),
]

# System
control_sismo = ctrl.ControlSystem(rules)

# -------------------------------
# Programmatic API + CLI + Notebook UI
# -------------------------------
def predict(acel, freq, dur, snr_db, dist_km=300.0):
    sim = ctrl.ControlSystemSimulation(control_sismo)   # fresh sim
    sim.input['aceleracion'] = float(acel)
    sim.input['frecuencia']  = float(freq)
    sim.input['duracion']    = float(dur)
    sim.input['snr']         = float(snr_db)
    sim.input['distancia']   = float(dist_km)
    sim.compute()
    val = sim.output['amenaza']
    if val < 2:    label = "No seismic activity"
    elif val < 4:  label = "Microtremor"
    elif val < 6:  label = "Mild earthquake"
    elif val < 8:  label = "Moderate earthquake"
    elif val < 9.5:label = "Strong earthquake"
    else:          label = "Destructive earthquake"
    return val, label

def run_cli():
    val_acel = leer_float("Enter ground acceleration (g, 0–1.5): ", 0.0, 1.5)
    val_freq = leer_float("Enter dominant frequency (Hz, 0–20): ", 0.0, 20.0)
    val_dur  = leer_float("Enter event duration (s, 0–60): ", 0.0, 60.0)
    val_dist = leer_float("Enter distance to plate boundary (km, 0–600) [Enter=300]: ",
                          0.0, 600.0, default=300.0, allow_blank=True)
    val_snr  = leer_float("Enter SNR (dB, 0–30): ", 0.0, 30.0)
    val, label = predict(val_acel, val_freq, val_dur, val_snr, val_dist)
    print(f"\n🔍 Seismic threat level: {val:.2f} / 10")
    print(f"➡️ Classification: {label}")

def notebook_ui():
    from IPython.display import display
    import ipywidgets as W
    a = W.FloatSlider(description='a (g)', min=0, max=1.5, step=0.01, value=0.6)
    f = W.FloatSlider(description='f (Hz)', min=0, max=20,  step=0.1, value=4.0)
    d = W.IntSlider(  description='dur (s)', min=0, max=60,  step=1,   value=20)
    s = W.FloatSlider(description='SNR (dB)', min=0, max=30, step=0.1, value=18.0)
    x = W.IntSlider(  description='dist (km)', min=0, max=600, step=1, value=300)
    out = W.Output()
    btn = W.Button(description='Compute', button_style='primary')
    def on_click(_):
        with out:
            out.clear_output()
            val, label = predict(a.value, f.value, d.value, s.value, x.value)
            print(f"Threat: {val:.2f} / 10 — {label}")
    btn.on_click(on_click)
    display(W.VBox([W.HBox([a, f]), W.HBox([d, s, x]), btn, out]))

# Entry point
if in_notebook():
    try:
        notebook_ui()          # shows sliders in the cell output area
    except Exception:
        val, label = predict(0.6, 4.0, 25, 18.0, 120.0)
        print(f"Demo → Threat: {val:.2f} / 10 | {label}")
else:
    run_cli()
